In [1]:
from CompressionMethods import static_quantization, distillation, pruning
import pandas as pd

/home/codespace/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Initialize all the required parameters

In [2]:
model = "adjohn1313/bert-base-finetuned"
model_type = "bert"
dataset = "sem_eval_2018_task_1"
subtask = "subtask5.english"

### 1. Static Quantization

In [3]:
staticQuantizationObject = static_quantization.staticQuantization(model_id=model, dataset_id=dataset, dataset_subsetid=subtask)
staticQuantizationObject.run_experiment()

Results of 4-bit Static Quantization:
{'accuracy': 0.27539503386004516,
 'mean_time': 0.02808135303902034,
 'method': '4bit-static-quantization',
 'size': 86.50099754333496}
####################################################################################################
Results of 8-bit Static Quantization:
{'accuracy': 0.26749435665914223,
 'mean_time': 0.10081183049264396,
 'method': '8bit-static-quantization',
 'size': 127.28224754333496}
####################################################################################################


Removed shared tensor {'bert.encoder.layer.4.attention.output.dense.weight_format', 'bert.encoder.layer.0.output.dense.weight_format', 'bert.encoder.layer.5.intermediate.dense.weight_format', 'bert.encoder.layer.10.attention.self.value.weight_format', 'bert.encoder.layer.1.attention.self.value.weight_format', 'bert.encoder.layer.2.attention.self.value.weight_format', 'bert.encoder.layer.2.attention.output.dense.weight_format', 'bert.encoder.layer.6.attention.self.value.weight_format', 'bert.encoder.layer.8.intermediate.dense.weight_format', 'bert.encoder.layer.9.attention.self.key.weight_format', 'bert.encoder.layer.1.output.dense.weight_format', 'bert.encoder.layer.5.output.dense.weight_format', 'bert.encoder.layer.6.output.dense.weight_format', 'bert.encoder.layer.10.attention.output.dense.weight_format', 'bert.encoder.layer.7.attention.self.key.weight_format', 'bert.pooler.dense.weight_format', 'bert.encoder.layer.8.attention.self.query.weight_format', 'bert.encoder.layer.2.output.d

### 2. Distillation

In [4]:
student_model = "distilbert/distilbert-base-uncased"
distillationObject = distillation.DistillationModule(teacher_model_id=model, student_model_id=student_model, 
                                                     dataset=dataset, sub_dataset=subtask)
distillationObject.run_experiment()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 886/886 [00:00<00:00, 14192.35 examples/s]
/home/codespace/.local/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,1.062500,0.344743,0.696940,0.797612,0.266366
2,0.400600,0.300171,0.698539,0.795519,0.278781
3,0.274900,0.283933,0.695500,0.792310,0.286682
4,0.229700,0.271652,0.696810,0.792373,0.284424
5,0.191000,0.264767,0.696130,0.792872,0.273138


Training time:  448.6524896621704
Results of Distillation:
{'accuracy': 0.2742663656884876,
 'mean_time': 0.006577949760729788,
 'method': 'Distillation (BERT Only)',
 'size': 255.44340133666992}
####################################################################################################


### 3. Pruning

In [5]:
pruningObject = pruning.PruneModel(model_id=model, dataset_id=dataset, dataset_subsetid=subtask)
pruningObject.run_experiment()

Results of Pruning:
{'method': 'Pruning', 'size': 417.6816825866699, 'mean_time': 0.012086537298714872, 'accuracy': 0.27200902934537247}
####################################################################################################


### Saving the results from all experiments

In [6]:
results = pd.DataFrame([
    staticQuantizationObject.results_4bit, 
    staticQuantizationObject.results_8bit, 
    distillationObject.results_distillation, 
    pruningObject.results
    ])
display(results.head())
model_name = model.split("/")[-1]
results.to_csv(f"{model_name}_results.csv", index=False)

,method,size,mean_time,accuracy
0,4bit-static-quantization,86.500998,0.028081,0.275395
1,8bit-static-quantization,127.282248,0.100812,0.267494
2,Distillation (BERT Only),255.443401,0.006578,0.274266
3,Pruning,417.681683,0.012087,0.272009
